![Tennis Top 10 Players](tennis-dashboard-cover.png){width=100% fig-align="center"}

---

In [ ]:
#| label: setup
#| code-summary: "Dashboard Setup"
#| output: false

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML, Markdown
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)

import plotly.io as pio
pio.templates.default = 'plotly_white'

# Load data
DATA_PATH = '../../data/top10'
atp_players = pd.read_csv(f'{DATA_PATH}/atp/atp_top10_players.csv')
atp_rankings = pd.read_csv(f'{DATA_PATH}/atp/atp_top10_rankings.csv')
atp_matches = pd.read_csv(f'{DATA_PATH}/atp/atp_top10_matches.csv')
wta_players = pd.read_csv(f'{DATA_PATH}/wta/wta_top10_players.csv')
wta_rankings = pd.read_csv(f'{DATA_PATH}/wta/wta_top10_rankings.csv')
wta_matches = pd.read_csv(f'{DATA_PATH}/wta/wta_top10_matches.csv')

# Process data
def parse_dob(dob):
    if pd.isna(dob): return None
    try: return datetime.strptime(str(int(dob)), '%Y%m%d')
    except: return None

def calculate_age(dob):
    if dob is None: return None
    return (datetime(2024, 12, 30) - dob).days / 365.25

for df in [atp_players, wta_players]:
    df['dob_parsed'] = df['dob'].apply(parse_dob)
    df['age'] = df['dob_parsed'].apply(calculate_age)
    df['full_name'] = df['name_first'] + ' ' + df['name_last']

atp_players['tour'], wta_players['tour'] = 'ATP', 'WTA'
atp_matches['tour'], wta_matches['tour'] = 'ATP', 'WTA'

# Parse dates
atp_rankings['date'] = pd.to_datetime(atp_rankings['ranking_date'].astype(str), format='%Y%m%d')
atp_rankings = atp_rankings.merge(atp_players[['player_id', 'full_name']], left_on='player', right_on='player_id')
wta_rankings['date'] = pd.to_datetime(wta_rankings['ranking_date'].astype(str), format='%Y%m%d')
wta_rankings = wta_rankings.merge(wta_players[['player_id', 'full_name']], left_on='player', right_on='player_id')

atp_matches['tourney_date'] = pd.to_datetime(atp_matches['tourney_date'].astype(str), format='%Y%m%d')
wta_matches['tourney_date'] = pd.to_datetime(wta_matches['tourney_date'].astype(str), format='%Y%m%d')

all_players = pd.concat([atp_players, wta_players], ignore_index=True)

# Quick Stats

Key metrics from our dataset covering the **Top 10 ATP and WTA players** as of December 30, 2024.

In [ ]:
#| label: kpi-cards
#| code-summary: "Generate KPI cards"

# Calculate key metrics
total_matches = len(atp_matches) + len(wta_matches)
atp_top_ranked = atp_players.loc[atp_rankings[atp_rankings['rank'] == 1]['player_id'].mode().iloc[0] == atp_players['player_id'], 'full_name'].iloc[0] if len(atp_rankings[atp_rankings['rank'] == 1]) > 0 else 'N/A'

# Weeks at #1 calculations
atp_weeks_1 = atp_rankings[atp_rankings['rank'] == 1].groupby('full_name').size().max()
wta_weeks_1 = wta_rankings[wta_rankings['rank'] == 1].groupby('full_name').size().max()

kpi_html = f'''
<div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 20px; margin: 20px 0;">
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 25px; border-radius: 15px; text-align: center; color: white; box-shadow: 0 4px 15px rgba(0,0,0,0.1);">
        <div style="font-size: 2.5em; font-weight: bold;">{total_matches:,}</div>
        <div style="font-size: 0.9em; opacity: 0.9;">Total Matches Analyzed</div>
    </div>
    <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 25px; border-radius: 15px; text-align: center; color: white; box-shadow: 0 4px 15px rgba(0,0,0,0.1);">
        <div style="font-size: 2.5em; font-weight: bold;">20</div>
        <div style="font-size: 0.9em; opacity: 0.9;">Elite Players Tracked</div>
    </div>
    <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 25px; border-radius: 15px; text-align: center; color: white; box-shadow: 0 4px 15px rgba(0,0,0,0.1);">
        <div style="font-size: 2.5em; font-weight: bold;">{atp_weeks_1}</div>
        <div style="font-size: 0.9em; opacity: 0.9;">ATP Most Weeks at #1</div>
    </div>
    <div style="background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); padding: 25px; border-radius: 15px; text-align: center; color: white; box-shadow: 0 4px 15px rgba(0,0,0,0.1);">
        <div style="font-size: 2.5em; font-weight: bold;">{wta_weeks_1}</div>
        <div style="font-size: 0.9em; opacity: 0.9;">WTA Most Weeks at #1</div>
    </div>
</div>
'''
display(HTML(kpi_html))

# Player Overview

::: {.panel-tabset}

## ATP Tour


In [ ]:
#| label: atp-overview
#| code-summary: "ATP Player Overview"

# Calculate records
def calculate_record(matches_df, players_df):
    records = []
    for _, player in players_df.iterrows():
        pid = player['player_id']
        wins = len(matches_df[matches_df['winner_id'] == pid])
        losses = len(matches_df[matches_df['loser_id'] == pid])
        total = wins + losses
        win_pct = (wins / total * 100) if total > 0 else 0
        records.append({
            'Player': player['full_name'],
            'Country': player['ioc'],
            'Age': round(player['age'], 1) if player['age'] else None,
            'Height': player['height'],
            'Wins': wins,
            'Losses': losses,
            'Win %': round(win_pct, 1)
        })
    return pd.DataFrame(records).sort_values('Win %', ascending=False)

atp_overview = calculate_record(atp_matches, atp_players)

# Create visualization
fig = go.Figure()
atp_sorted = atp_overview.sort_values('Win %', ascending=True)
colors = ['#FFD700' if x >= 80 else '#2196F3' for x in atp_sorted['Win %']]

fig.add_trace(go.Bar(
    x=atp_sorted['Win %'],
    y=atp_sorted['Player'],
    orientation='h',
    marker_color=colors,
    text=[f"{x}%" for x in atp_sorted['Win %']],
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Win Rate: %{x}%<br>Record: %{customdata[0]}-%{customdata[1]}<extra></extra>',
    customdata=atp_sorted[['Wins', 'Losses']].values
))

fig.add_vline(x=80, line_dash='dash', line_color='gold', annotation_text='Elite (80%)')

fig.update_layout(
    title=dict(text='ATP Top 10 Career Win Rate', font=dict(size=20)),
    height=450,
    xaxis_title='Win Percentage',
    margin=dict(l=150, r=50, t=60, b=50),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print("\n📊 Full Player Data (sortable):")
show(atp_overview, classes="display compact", scrollY="300px")

## WTA Tour


In [ ]:
#| label: wta-overview
#| code-summary: "WTA Player Overview"

wta_overview = calculate_record(wta_matches, wta_players)

fig = go.Figure()
wta_sorted = wta_overview.sort_values('Win %', ascending=True)
colors = ['#FFD700' if x >= 80 else '#E91E63' for x in wta_sorted['Win %']]

fig.add_trace(go.Bar(
    x=wta_sorted['Win %'],
    y=wta_sorted['Player'],
    orientation='h',
    marker_color=colors,
    text=[f"{x}%" for x in wta_sorted['Win %']],
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Win Rate: %{x}%<br>Record: %{customdata[0]}-%{customdata[1]}<extra></extra>',
    customdata=wta_sorted[['Wins', 'Losses']].values
))

fig.add_vline(x=80, line_dash='dash', line_color='gold', annotation_text='Elite (80%)')

fig.update_layout(
    title=dict(text='WTA Top 10 Career Win Rate', font=dict(size=20)),
    height=450,
    xaxis_title='Win Percentage',
    margin=dict(l=150, r=50, t=60, b=50),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print("\n📊 Full Player Data (sortable):")
show(wta_overview, classes="display compact", scrollY="300px")

:::

# Rankings History

Track how these players have risen through the rankings. Use the **range slider** and **year buttons** to zoom into specific time periods. Click on player names in the legend to toggle visibility.

::: {.panel-tabset}

## ATP Rankings

In [ ]:
#| label: atp-rankings
#| code-summary: "ATP Rankings History"

fig = px.line(
    atp_rankings.sort_values('date'),
    x='date', y='rank', color='full_name',
    labels={'date': 'Year', 'rank': 'Ranking', 'full_name': 'Player'},
    hover_data={'date': '|%B %Y', 'rank': True}
)

fig.update_yaxes(autorange='reversed', range=[150, 1])
fig.add_hline(y=10, line_dash='dash', line_color='gray', opacity=0.5)
fig.add_hline(y=1, line_dash='dash', line_color='gold', opacity=0.7)

fig.update_layout(
    title=dict(text='ATP Rankings Journey', font=dict(size=20)),
    height=550,
    legend=dict(orientation='h', yanchor='bottom', y=-0.25, xanchor='center', x=0.5),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1Y', step='year', stepmode='backward'),
                dict(count=5, label='5Y', step='year', stepmode='backward'),
                dict(count=10, label='10Y', step='year', stepmode='backward'),
                dict(step='all', label='All')
            ]),
            bgcolor='rgba(150,150,150,0.1)',
            activecolor='#667eea'
        ),
        rangeslider=dict(visible=True, thickness=0.05),
        type='date'
    ),
    plot_bgcolor='rgba(0,0,0,0)',
    hovermode='x unified'
)
fig.show()

## WTA Rankings

In [ ]:
#| label: wta-rankings
#| code-summary: "WTA Rankings History"

fig = px.line(
    wta_rankings.sort_values('date'),
    x='date', y='rank', color='full_name',
    labels={'date': 'Year', 'rank': 'Ranking', 'full_name': 'Player'},
    hover_data={'date': '|%B %Y', 'rank': True}
)

fig.update_yaxes(autorange='reversed', range=[150, 1])
fig.add_hline(y=10, line_dash='dash', line_color='gray', opacity=0.5)
fig.add_hline(y=1, line_dash='dash', line_color='gold', opacity=0.7)

fig.update_layout(
    title=dict(text='WTA Rankings Journey', font=dict(size=20)),
    height=550,
    legend=dict(orientation='h', yanchor='bottom', y=-0.25, xanchor='center', x=0.5),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1Y', step='year', stepmode='backward'),
                dict(count=5, label='5Y', step='year', stepmode='backward'),
                dict(step='all', label='All')
            ]),
            bgcolor='rgba(150,150,150,0.1)',
            activecolor='#E91E63'
        ),
        rangeslider=dict(visible=True, thickness=0.05),
        type='date'
    ),
    plot_bgcolor='rgba(0,0,0,0)',
    hovermode='x unified'
)
fig.show()

:::

# Surface Performance

Tennis is played on different surfaces—Hard, Clay, and Grass—and players often excel on specific surfaces. Explore the heatmaps and radar charts below.

::: {.panel-tabset}

## Performance Heatmaps

In [ ]:
#| label: surface-heatmap
#| code-summary: "Surface Performance Heatmaps"

def surface_record(matches_df, players_df):
    results = []
    for _, player in players_df.iterrows():
        pid, name = player['player_id'], player['full_name']
        row = {'Player': name}
        for surface in ['Hard', 'Clay', 'Grass']:
            surface_matches = matches_df[matches_df['surface'] == surface]
            wins = len(surface_matches[surface_matches['winner_id'] == pid])
            losses = len(surface_matches[surface_matches['loser_id'] == pid])
            total = wins + losses
            row[surface] = round(wins / total * 100, 1) if total >= 10 else None
        results.append(row)
    return pd.DataFrame(results)

atp_surface = surface_record(atp_matches, atp_players)
wta_surface = surface_record(wta_matches, wta_players)

fig = make_subplots(rows=1, cols=2, subplot_titles=('ATP Surface Mastery', 'WTA Surface Mastery'), horizontal_spacing=0.12)

atp_heat = atp_surface.set_index('Player')[['Hard', 'Clay', 'Grass']].dropna(how='all')
wta_heat = wta_surface.set_index('Player')[['Hard', 'Clay', 'Grass']].dropna(how='all')

fig.add_trace(go.Heatmap(
    z=atp_heat.values, x=atp_heat.columns, y=atp_heat.index,
    colorscale='Blues', zmin=50, zmax=95,
    text=[[f'{v:.1f}%' if pd.notna(v) else '' for v in row] for row in atp_heat.values],
    texttemplate='%{text}', textfont=dict(size=11),
    hovertemplate='%{y}<br>%{x}: %{z:.1f}%<extra></extra>',
    showscale=False
), row=1, col=1)

fig.add_trace(go.Heatmap(
    z=wta_heat.values, x=wta_heat.columns, y=wta_heat.index,
    colorscale='RdPu', zmin=50, zmax=95,
    text=[[f'{v:.1f}%' if pd.notna(v) else '' for v in row] for row in wta_heat.values],
    texttemplate='%{text}', textfont=dict(size=11),
    hovertemplate='%{y}<br>%{x}: %{z:.1f}%<extra></extra>',
    colorbar=dict(title='Win %', x=1.02, ticksuffix='%')
), row=1, col=2)

fig.update_layout(height=500, title_text='Win Percentage by Surface (min. 10 matches)')
fig.show()

## Player Profiles (Radar)

In [ ]:
#| label: radar-charts
#| code-summary: "Surface Profile Radar Charts"

# ATP Radar
atp_top = ['Novak Djokovic', 'Carlos Alcaraz', 'Jannik Sinner', 'Alexander Zverev']
categories = ['Hard', 'Clay', 'Grass', 'Hard']

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'polar'}]],
                    subplot_titles=('ATP Top Players', 'WTA Top Players'))

colors_atp = px.colors.qualitative.Set2[:4]
for i, player in enumerate(atp_top):
    pdata = atp_surface[atp_surface['Player'] == player]
    if not pdata.empty:
        values = [pdata['Hard'].values[0], pdata['Clay'].values[0], pdata['Grass'].values[0], pdata['Hard'].values[0]]
        fig.add_trace(go.Scatterpolar(
            r=values, theta=categories, name=player.split()[-1],
            fill='toself', opacity=0.6, line=dict(color=colors_atp[i])
        ), row=1, col=1)

# WTA Radar
wta_top = ['Iga Swiatek', 'Aryna Sabalenka', 'Coco Gauff', 'Elena Rybakina']
colors_wta = px.colors.qualitative.Pastel[:4]
for i, player in enumerate(wta_top):
    pdata = wta_surface[wta_surface['Player'] == player]
    if not pdata.empty:
        values = [pdata['Hard'].values[0], pdata['Clay'].values[0], pdata['Grass'].values[0], pdata['Hard'].values[0]]
        fig.add_trace(go.Scatterpolar(
            r=values, theta=categories, name=player.split()[-1],
            fill='toself', opacity=0.6, line=dict(color=colors_wta[i])
        ), row=1, col=2)

fig.update_polars(radialaxis=dict(visible=True, range=[40, 100]))
fig.update_layout(height=450, showlegend=True,
                  legend=dict(orientation='h', yanchor='bottom', y=-0.15, xanchor='center', x=0.5))
fig.show()

:::

# Head-to-Head Matrix

How do the top 10 players fare against each other? Green indicates a winning record, red indicates a losing record.

::: {.panel-tabset}

## ATP H2H

In [ ]:
#| label: atp-h2h
#| code-summary: "ATP Head-to-Head Matrix"

def head_to_head_matrix(matches_df, players_df):
    player_ids = players_df['player_id'].tolist()
    player_names = players_df['full_name'].tolist()
    h2h_matches = matches_df[(matches_df['winner_id'].isin(player_ids)) & (matches_df['loser_id'].isin(player_ids))]
    
    matrix_display = pd.DataFrame(index=player_names, columns=player_names)
    matrix_numeric = pd.DataFrame(index=player_names, columns=player_names, dtype=float)
    
    for i, p1_id in enumerate(player_ids):
        for j, p2_id in enumerate(player_ids):
            if i == j:
                matrix_display.iloc[i, j], matrix_numeric.iloc[i, j] = '-', np.nan
            else:
                wins = len(h2h_matches[(h2h_matches['winner_id'] == p1_id) & (h2h_matches['loser_id'] == p2_id)])
                losses = len(h2h_matches[(h2h_matches['winner_id'] == p2_id) & (h2h_matches['loser_id'] == p1_id)])
                total = wins + losses
                matrix_display.iloc[i, j] = f"{wins}-{losses}" if total > 0 else '0-0'
                matrix_numeric.iloc[i, j] = (wins / total * 100) if total > 0 else 50
    
    return matrix_display, matrix_numeric, len(h2h_matches)

atp_h2h_display, atp_h2h_numeric, atp_h2h_count = head_to_head_matrix(atp_matches, atp_players)
short_names = [n.split()[-1] for n in atp_h2h_numeric.index]

fig = go.Figure(data=go.Heatmap(
    z=atp_h2h_numeric.values, x=short_names, y=short_names,
    colorscale='RdYlGn', zmin=0, zmax=100,
    text=atp_h2h_display.values, texttemplate='%{text}',
    hovertemplate='<b>%{y} vs %{x}</b><br>Record: %{text}<br>Win Rate: %{z:.0f}%<extra></extra>',
    colorbar=dict(title='Win %', ticksuffix='%')
))

fig.update_layout(
    title=f'ATP Head-to-Head ({atp_h2h_count} matches between Top 10)',
    height=550, xaxis_title='Opponent', yaxis_title='Player',
    yaxis=dict(autorange='reversed')
)
fig.show()

## WTA H2H

In [ ]:
#| label: wta-h2h
#| code-summary: "WTA Head-to-Head Matrix"

wta_h2h_display, wta_h2h_numeric, wta_h2h_count = head_to_head_matrix(wta_matches, wta_players)
short_names_wta = [n.split()[-1] for n in wta_h2h_numeric.index]

fig = go.Figure(data=go.Heatmap(
    z=wta_h2h_numeric.values, x=short_names_wta, y=short_names_wta,
    colorscale='RdYlGn', zmin=0, zmax=100,
    text=wta_h2h_display.values, texttemplate='%{text}',
    hovertemplate='<b>%{y} vs %{x}</b><br>Record: %{text}<br>Win Rate: %{z:.0f}%<extra></extra>',
    colorbar=dict(title='Win %', ticksuffix='%')
))

fig.update_layout(
    title=f'WTA Head-to-Head ({wta_h2h_count} matches between Top 10)',
    height=550, xaxis_title='Opponent', yaxis_title='Player',
    yaxis=dict(autorange='reversed')
)
fig.show()

:::

# Serve Analysis

The serve is crucial in tennis. Explore ace rates, double faults, and serve effectiveness.

In [ ]:
#| label: serve-analysis
#| code-summary: "Serve Statistics Analysis"

def calculate_serve_stats(matches_df, players_df):
    results = []
    for _, player in players_df.iterrows():
        pid, name = player['player_id'], player['full_name']
        wins = matches_df[matches_df['winner_id'] == pid][['w_ace', 'w_df', 'w_1stIn', 'w_svpt', 'w_1stWon', 'w_2ndWon']].copy()
        wins.columns = ['aces', 'df', '1st_in', 'svpt', '1st_won', '2nd_won']
        losses = matches_df[matches_df['loser_id'] == pid][['l_ace', 'l_df', 'l_1stIn', 'l_svpt', 'l_1stWon', 'l_2ndWon']].copy()
        losses.columns = ['aces', 'df', '1st_in', 'svpt', '1st_won', '2nd_won']
        all_m = pd.concat([wins, losses]).dropna()
        if len(all_m) > 0:
            results.append({
                'Player': name,
                'Aces/Match': round(all_m['aces'].mean(), 1),
                'DFs/Match': round(all_m['df'].mean(), 1),
                '1st Serve %': round((all_m['1st_in'] / all_m['svpt']).mean() * 100, 1),
                '1st Win %': round((all_m['1st_won'] / all_m['1st_in']).mean() * 100, 1)
            })
    return pd.DataFrame(results)

atp_serve = calculate_serve_stats(atp_matches, atp_players)
atp_serve['Tour'] = 'ATP'
wta_serve = calculate_serve_stats(wta_matches, wta_players)
wta_serve['Tour'] = 'WTA'
all_serve = pd.concat([atp_serve, wta_serve], ignore_index=True)

fig = px.scatter(
    all_serve, x='Aces/Match', y='DFs/Match', color='Tour', size='1st Win %',
    hover_name='Player', hover_data={'1st Serve %': True, '1st Win %': True},
    title='Serve Performance: Aces vs Double Faults',
    color_discrete_map={'ATP': '#2196F3', 'WTA': '#E91E63'},
    labels={'Aces/Match': 'Average Aces per Match', 'DFs/Match': 'Average Double Faults per Match'}
)

for _, row in all_serve.iterrows():
    fig.add_annotation(x=row['Aces/Match'], y=row['DFs/Match'], text=row['Player'].split()[-1],
                       showarrow=False, yshift=12, font=dict(size=9))

fig.update_traces(marker=dict(line=dict(width=1, color='white')))
fig.update_layout(height=500, legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5))
fig.show()

print("\n📊 Full Serve Statistics:")
show(all_serve, classes="display compact", scrollY="300px")

# Yearly Performance

Track player performance year over year with interactive range selection.

::: {.panel-tabset}

## ATP Yearly Wins

In [ ]:
#| label: atp-yearly
#| code-summary: "ATP Yearly Performance"

def yearly_wins(matches_df, players_df, start_year=2018):
    matches_df = matches_df[matches_df['tourney_date'].dt.year >= start_year].copy()
    matches_df['year'] = matches_df['tourney_date'].dt.year
    results = []
    for _, player in players_df.iterrows():
        pid, name = player['player_id'], player['full_name']
        for year in range(start_year, 2025):
            year_matches = matches_df[matches_df['year'] == year]
            wins = len(year_matches[year_matches['winner_id'] == pid])
            losses = len(year_matches[year_matches['loser_id'] == pid])
            total = wins + losses
            results.append({'Player': name, 'Year': year, 'Wins': wins, 'Losses': losses,
                          'Win %': round(wins / total * 100, 1) if total > 0 else 0})
    return pd.DataFrame(results)

atp_yearly = yearly_wins(atp_matches, atp_players)

fig = px.line(atp_yearly, x='Year', y='Wins', color='Player', markers=True,
              hover_data={'Win %': True, 'Losses': True},
              title='ATP Yearly Match Wins (2018-2024)')

fig.update_layout(
    height=500,
    legend=dict(orientation='h', yanchor='bottom', y=-0.3, xanchor='center', x=0.5),
    xaxis=dict(rangeslider=dict(visible=True, thickness=0.05), tickmode='linear', dtick=1)
)
fig.show()

## WTA Yearly Wins

In [ ]:
#| label: wta-yearly
#| code-summary: "WTA Yearly Performance"

wta_yearly = yearly_wins(wta_matches, wta_players)

fig = px.line(wta_yearly, x='Year', y='Wins', color='Player', markers=True,
              hover_data={'Win %': True, 'Losses': True},
              title='WTA Yearly Match Wins (2018-2024)')

fig.update_layout(
    height=500,
    legend=dict(orientation='h', yanchor='bottom', y=-0.3, xanchor='center', x=0.5),
    xaxis=dict(rangeslider=dict(visible=True, thickness=0.05), tickmode='linear', dtick=1)
)
fig.show()

:::

# Rankings Race Animation

Watch how rankings have evolved from 2020 to 2024. Press **Play** to animate!

In [ ]:
#| label: rankings-animation
#| code-summary: "Animated Rankings Race"

atp_rankings['year'] = atp_rankings['date'].dt.year
atp_yearly_rank = atp_rankings[atp_rankings['year'] >= 2020].groupby(['full_name', 'year']).agg(
    best_rank=('rank', 'min')).reset_index()

fig = px.bar(
    atp_yearly_rank.sort_values(['year', 'best_rank']),
    x='best_rank', y='full_name', color='full_name',
    animation_frame='year', orientation='h', range_x=[0, 50],
    title='ATP Best Ranking by Year (2020-2024) - Press Play!',
    labels={'best_rank': 'Best Ranking', 'full_name': 'Player'}
)

fig.update_layout(
    height=500, showlegend=False,
    yaxis={'categoryorder': 'total ascending'},
    xaxis={'autorange': 'reversed'}
)
fig.show()

# Key Insights

::: {.callout-note}
## ATP Highlights
- **Novak Djokovic** holds the record with 377 weeks at #1
- **Jannik Sinner** and **Carlos Alcaraz** represent the new generation dominating the tour
- **Alexander Zverev** leads in aces per match among the top 10
:::

::: {.callout-tip}
## WTA Highlights
- **Iga Swiatek** dominated with 120 weeks at #1 and exceptional clay court performance
- **Aryna Sabalenka** has been the most consistent performer in 2024
- **Coco Gauff** is the youngest in the top 10 at just 20.8 years
:::

---

*Data source: [Tennis Abstract](https://github.com/JeffSackmann) by Jeff Sackmann (CC BY-NC-SA 4.0)*